In [68]:
import openai
import requests
import tiktoken
from tqdm.auto import tqdm
from openai import OpenAI
from elasticsearch import Elasticsearch

### Q1. Running Elastic

In [69]:
es_client = Elasticsearch('http://localhost:9200')
es_client.info()

ObjectApiResponse({'name': 'e385745e689d', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'CUmt7zkWTy2ZcyaPOs_7lg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [3]:
index_name = "course-questions"
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [4]:
def get_documents() -> list:
    docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    documents = []

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            documents.append(doc)
    return documents

documents = get_documents()

## Q2. Indexing the data

In [5]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:23<00:00, 40.26it/s]


## Q3. Searching

In [70]:
query = "How do I execute a command in a running docker container?"


search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}
response = es_client.search(index=index_name, body=search_query)

In [71]:
response['hits']['hits'][0]['_score']

75.54128

## Q4. Filtering

In [72]:
query = "How do I execute a command in a running docker container?"


search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}
response = es_client.search(index=index_name, body=search_query)

In [73]:
response['hits']['hits'][2]['_source']['question']

'How do I copy files from a different folder into docker container’s working directory?'

## Q5. Building a prompt

In [50]:
records = response['hits']['hits']
context_docs = []
for record in records:
    context_docs.append(record['_source'])

In [51]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [52]:
context_elements = []
for doc in context_docs:
    doc_context = f"""
Q: {doc['question']}
A: {doc['text']}
""".strip()
    context_elements.append(doc_context)

context = '\n\n'.join(context_elements)
print(context)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [53]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [54]:
prompt = f"""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {query}

CONTEXT:
{context}
""".strip()

print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

In [55]:
len(prompt)

1462

## Q6. Tokens

In [65]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [66]:
tokenized_prompt = encoding.encode(prompt)
len(tokenized_prompt)

322

## generating the answer

In [74]:
client = OpenAI()
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[{"role": "user", "content": prompt}]
)
answer = response.choices[0].message.content

'To execute a command in a running docker container, you need to first find the container-id by running `docker ps`. Then you can execute a bash command in the specific container using the command `docker exec -it <container-id> bash`.'

## calculating the costs

In [78]:
input_cost = 0.005
output_cost = 0.015

tokenized_prompts = 150 * 1000 * input_cost / 1000
tokenized_answers = 250 * 1000 * output_cost / 1000
price = tokenized_prompts + tokenized_answers
print(f"Price for 1000 requests will be {price}$")

Price for 1000 requests will be 4.5$
